In [1]:
from __future__ import print_function
from tornado.platform.twisted import install
install()
from twisted.internet import reactor

In [2]:
reactor.callLater(5, print, "hello")

hello


In [3]:
from twisted.internet import defer

In [4]:
def slowmult(a, b):
    d = defer.Deferred()
    reactor.callLater(2, d.callback, a*b)
    return d

In [5]:
slowmult(4, 5).addCallback(print)

20


In [6]:
d = slowmult(2, 3)
d.addCallback(slowmult, 4)
d.addCallback(print)

24
